In [2]:
import os
print(os.getcwd())

C:\Users\em18921\Documents\Sentinels_indicator\Codes_folder


In [4]:
import glob
import os
import pandas as pd
import numpy as np
from scipy.stats import zscore
from statsmodels.tsa.seasonal import STL
import re

# Function to create weekly change in SVI files

In [ ]:
def weekly_change_svi(df):
    # Sort the dataframe by year, month, day and groupby keyword_name
    df_sorted = df.sort_values(by=['keyword_name', 'Year', 'Month', 'Day'])

    # Groupby keyword name
    grouped = df_sorted.groupby('keyword_name')

    Delta_SVI = df_sorted.copy()

    Delta_SVI[f"delta_SVI"] = grouped['SVI'].diff()

    return Delta_SVI

# Creating weekly change in SVI for geopolitics file

In [ ]:
# Step 1: Import the dataset
input_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\geonews_final_dataset.csv'
df_geo = pd.read_csv(input_file_path, low_memory=False)

# Step 2: Replace '<1' value to 0
df_geo['SVI'] = df_geo['SVI'].replace('<1', 0)

# Step 3: Convert SVI column to numeric
df_geo['SVI'] = pd.to_numeric(df_geo['SVI'], errors='coerce')

# Step 4: Remove rows with less than 10 SVIs
df_geo = df_geo[df_geo['SVI'] >= 0]

# Step 2: Apply the function weekly change SVI to the above dataset
df_weekly_change_geo = weekly_change_svi(df_geo)

# Step 3: Output the answer file
#output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_output_folder_16Dec\geopoltiks_cal_1.csv'
output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\cal_1_geonews.csv'
df_weekly_change_geo.to_csv(output_file_path, index=False)
df_weekly_change_geo

# Function to winsorize delta_SVI of each query at 2.5% level at each tail

In [ ]:
def winsorize_data(df, lower=0.025, upper=0.975):
    df_sorted = df.sort_values(by=['keyword_name', 'Year', 'Month', 'Day'])

    win_df = df_sorted.copy()

    # Group the data by keyword_name
    grouped = df_sorted.groupby('keyword_name')

    lower_quantile = grouped['delta_SVI'].quantile(lower).reset_index(name='lower_quantile')
    upper_quantile = grouped['delta_SVI']. quantile(upper).reset_index(name='upper_quantile')

    # Merge the quantile back to the dataframe
    win_df = win_df.merge(lower_quantile, on='keyword_name')
    win_df = win_df.merge(upper_quantile, on='keyword_name')

    # winsorize delta SVI
    win_df['winsorize_delta'] = win_df['delta_SVI'].clip(lower=win_df['lower_quantile'], upper=win_df['upper_quantile'])

    return win_df

# Winsorizing delta_SVI values for Geo Index data

In [ ]:
# Step 1: Import the dataset
input_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\cal_1_geonews.csv'
df_geo = pd.read_csv(input_file_path)

# Step 2: Call the winsorize function and winsorize the dataser
win_df_Geo = winsorize_data(df_geo)

# Step 3: Output the dataset
#output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_output_folder_16Dec\geopoltiks_cal_2.csv'
output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\cal_2_geonews.csv'
win_df_Geo.to_csv(output_file_path, index=False)
win_df_Geo

# function to create monthly dummies for regression for seasonality

In [ ]:
def create_dummies_var(df, drop_first=True, drop_last=False):
    # get unique names of the month
    month_map = {1:'jan', 2:'feb', 3:'mar', 4:'apr', 5:'may', 6:'jun',
                 7:'jul', 8:'aug', 9:'sep', 10:'oct', 11:'nov', 12:'dec'}

    df['month_name'] = df['Month'].map(month_map)

    unique_categories = df['month_name'].unique()

    print(unique_categories)

    # Determine whats need to be dropped, drop_first or drop_last
    if drop_first == True:
        categories_to_encode = unique_categories[1:]
    if drop_last == True:
        categories_to_encode = unique_categories[:-1]
    else:
        categories_to_encode = unique_categories

    # Create a dictionary to hold dummy data
    dummy_data = {}

    for category in categories_to_encode:
        dummy_data[f"dummy_{category}"] = (df['month_name'] == category).astype(int)

    # create a dataframe from dummy_data
    dummies_df = pd.DataFrame(dummy_data, index=df.index)

    result_df = pd.concat([df, dummies_df], axis=1)

    return result_df

# Create monthly dummies data values for Geo Index data

In [ ]:
# Step 1: Import the dataset
input_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\cal_2_geonews.csv'
df_Google = pd.read_csv(input_file_path)

# Step 2: Call the function to create dummy monthly values
df_Google_dummy = create_dummies_var(df_Google)

# Step 3: Output the dataset
#output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_output_folder_16Dec\geopoltiks_cal_3.csv'
output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\cal_3_geonews.csv'
df_Google_dummy.to_csv(output_file_path, index=False)
df_Google_dummy

# Regression of monthly dummies on delta SVI

## Creating function for regression on monthly dummies

In [ ]:
import statsmodels.api as sm

def regression_dummies(df, cols_fr_regression_X):
    predicted_value_df = df.copy()
    residual_value_df = df.copy()

    # Creating date column to create index
    #predicted_value_df['date_derived'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    #residual_value_df['date_derived'] = pd.to_datetime(df[['Year', 'Month', 'Day']])

    y = df['winsorize_delta']

    # Define independent variable
    X = df[cols_fr_regression_X]

    # Add a constant term for independent variables( for the intercept)
    X = sm.add_constant(X)

    # Fit the regression model
    model = sm.OLS(y, X).fit()

    # Store predicted values
    predicted_value_df['winsorize_delta' + '_predicted'] = model.predict()

    # Store residual values
    residual_value_df['winsorize_delta' + '_residual'] = model.resid

    return predicted_value_df, residual_value_df

## Run regression for seasonality

In [ ]:
# Step 1: Import the file
input_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\cal_3_geonews.csv'
df_geo= pd.read_csv(input_file_path)

# Step 2: Initialize an empty list to hold columns for which seasonality has to be calculated or X axis
cols_fr_regression_X = ['week_number', 'dummy_feb', 'dummy_mar', 'dummy_apr', 'dummy_may', 
                        'dummy_jun', 'dummy_jul', 'dummy_aug', 'dummy_sep', 'dummy_oct', 'dummy_nov', 'dummy_dec']

# Step 3: Check for funny values in the dataset
for col in df_geo.columns:
    if df_geo['winsorize_delta'].isin([np.inf, -np.inf]).any() or df_geo[col].isnull().any():
        print(f"Funny value which will fail regression found in column : {col}")
    else:
        print(f"Funny value which will fail regression not found anywhere")

df_geo = df_geo.dropna()
df_geo.shape[0]
print("Independent variable: ", cols_fr_regression_X)

In [ ]:
# Calling function to perform seasonality regression
predicted_value_df_geo, residual_value_df_geo = regression_dummies(df_geo, cols_fr_regression_X)

# Output the dataset
#output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_output_folder_16Dec\geopoltiks_cal_predicted.csv'
#predicted_value_df_geo.to_csv(output_file_path, index=False)
predicted_value_df_geo

# Output the dataset
#output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_output_folder_16Dec\geopoltiks_cal_residual.csv'
output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\cal_residual_geonews.csv'
residual_value_df_geo.to_csv(output_file_path, index=False)
residual_value_df_geo

# Normalize residuals using standard deviation

In [ ]:
# Function to normalize residuals

def normalize_residual(df):
    normalize_residual_df = df.copy()
    normalize_residual_df = normalize_residual_df[normalize_residual_df['SVI'] > 1]

    normalize_residual_df['mean_residual'] = df.groupby('keyword_name')['winsorize_delta_residual'].transform('mean')
    normalize_residual_df['std_residual'] = df.groupby('keyword_name')['winsorize_delta_residual'].transform('std')

    normalize_residual_df[f"normalize_residual"] = (normalize_residual_df['winsorize_delta_residual'] - normalize_residual_df['mean_residual']) / normalize_residual_df['std_residual']

    return normalize_residual_df

# Calculating normalize residual values

In [ ]:
# Step 1: Import the file
input_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\cal_residual_geonews.csv'
df_geo = pd.read_csv(input_file_path)

# Step 2: Call the function
df_norm_residual_geo = normalize_residual(df_geo)

# Step 3: Output the dataset
#output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_output_folder_16Dec\geopoltiks_cal_4.csv'
output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\cal_4_geonews.csv'
df_norm_residual_geo.to_csv(output_file_path, index=False)
df_norm_residual_geo

# Importing snp 500 data.
## Making year and month column in it and merging with residual data calculated.

In [ ]:
input_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_output_folder_16Dec\snp500_data_vol.csv'
df_index_data = pd.read_csv(input_file_path)

df_index_data.drop(columns=['date'], inplace=True)  

input_file_path_2 = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\cal_4_geonews.csv'
df_main_data = pd.read_csv(input_file_path_2)

df_merged = pd.merge(df_main_data, df_index_data, on=['time_series_period_week'], how='left')
df_merged.dtypes

# Step 3: Output the dataset
output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\cal_5_geonews.csv'
df_merged.to_csv(output_file_path, index=False)
df_merged


# Function for rolling backward regression

# Running regression of keyword_name with GPR index

### This code is for cumulative regression

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

def expanding_rolling_regression(df):
    # Create a copy of the DataFrame to store results
    results_df = df.copy()
    results_df['t_stat'] = np.nan
    #results_df['date'] = pd.to_datetime(results_df['date'], format='%d/%m/%Y', errors='coerce')
    results_df['date_calculated'] = pd.to_datetime(df[['Year', 'Month', 'Day']])

    # Loop through each keyword group
    for keyword, group in results_df.groupby('keyword_name'):
        # Sort the group by time series period (to ensure expanding regression is done correctly)
        group = group.sort_values('time_series_period_week')

        # Iterate through each year to get the last day in June and December
        for year in range(group['date_calculated'].dt.year.min(), group['date_calculated'].dt.year.max() + 1):
            # Get the last day in June for the current year
            last_day_june = group[(group['date_calculated'].dt.year == year) & (group['date_calculated'].dt.month == 6)].tail(1)
            # Get the last day in December for the current year
            last_day_december = group[(group['date_calculated'].dt.year == year) & (group['date_calculated'].dt.month == 12)].tail(1)

            # Concatenate the last days for June and December
            last_weeks = pd.concat([last_day_june, last_day_december])

            # Loop through the filtered last weeks and perform regression for each date
            for idx, row in last_weeks.iterrows():
                current_date = row['date_calculated']

                # Define the expanding window up to the current date
                historical_data = group[group['date_calculated'] <= current_date]

                # Skip iteration if historical data is empty
                if historical_data.empty:
                    continue

                print(f"No of rows for {keyword} on {current_date} is {historical_data.shape[0]}")
                
                # Define Y (dependent variable) and X (independent variable)
                Y = historical_data['snp_Volatility']
                X = historical_data['normalize_residual']

                # Add a constant to X
                X = sm.add_constant(X)

                # Align Y and X
                Y, X = Y.align(X, join='inner')

                # Run the regression if there are enough data points
                if len(Y) > 1:  # Ensure there are at least two points for regression
                    model = sm.OLS(Y, X).fit()

                    # Extract t-statistic for the coefficient of 'normalize_residual'
                    t_stat_value = model.tvalues['normalize_residual'] if 'normalize_residual' in model.tvalues else np.nan

                    # Update the t_stat value in results_df for the current date and keyword
                    results_df.loc[idx, 't_stat'] = t_stat_value

    return results_df

# Running regression for GPR index

In [ ]:
# Step 1: Input file path of S&P 500 retruns
input_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\cal_5_geonews.csv'
df = pd.read_csv(input_file_path)

# Step 2: Call the function
df_geo = expanding_rolling_regression(df)

# Step 3: Output the dataset
output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\cal_6_geonews.csv'
df_geo.to_csv(output_file_path, index=False)
df_geo

# Creating t_stat_rank for GPR index sentiment data by ascending order

In [ ]:
import pandas as pd

# Step 1: Input file path of S&P 500 retruns
input_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\cal_6_geonews.csv'
df = pd.read_csv(input_file_path, low_memory=False)

# Step 2: Creating t-stat rank
df = df.sort_values(['Year', 'Month', 'Day']).reset_index(drop=True)

# Rank the t-stats values for each year and month
df.loc[df['Month'].isin([6, 12]), 'rank'] = df[df['Month'].isin([6, 12])].groupby(['Year', 'Month'])['t_stat'].rank(ascending=True, method='first')

# Step 4: Print the range of rank for each keyword
df = df.sort_values(['Year', 'Month']).reset_index(drop=True)

# range of rank
df_rank = df[(df['rank'] >= 1) & (df['rank'] <= 250)] 
grouped = df_rank.groupby(["Year", "Month"])

for (Year, Month), group in grouped:
    print(f"Rankings for: Year: {Year} and Month: {Month} and Total rank : {group['rank'].count()}")
    
# Step 5: Output the dataset
output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\cal_7_geonews.csv'
df.to_csv(output_file_path, index=False)
df

# Auto populate the last calculated ranking for each keyword

In [ ]:
# Step 1: Input file path of S&P 500 retruns
input_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\cal_7_geonews.csv'
df = pd.read_csv(input_file_path, low_memory=False)

# Step 2: Sort the dataset by keyword_name, year, month and day
df = df.sort_values(['keyword_name', 'Year', 'Month'])

# Step 3: Drag the last value till the next value for ranking. Eg, For june 2004, the ranking should be auto populated till the next value. i.e. December 2004
# Using forward fill process
df['rank'] = df.groupby(['keyword_name'])['rank'].fillna(method='ffill')

# Step 4: Output the dataset
output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\cal_8_geonews.csv'
df.to_csv(output_file_path, index=False)
df

# Sentiment = {(AVG of top 30 ASVI) - (AVG of bottom 30 ASVI)}

In [ ]:
def sentiment_cals(df):
    df = df.sort_values(['Year', 'Month', 'Day'])

    # Initialize lists to hold the sums
    grouped_weekly = df.groupby(['Year', 'Month', 'Day'])
    grouped_monthly = df.groupby(['Year', 'Month'])
                                 
    def calculate_top_30(group):
        top_30_sum = group.nlargest(30, 'rank')['normalize_residual'].mean()
        bottom_30_avg = group.nsmallest(30, 'rank')['normalize_residual'].mean()

        sentiment_indicator = (top_30_sum - bottom_30_avg)
        
        sentiment_indicator = sentiment_indicator
        
        return sentiment_indicator
        
    sentiment_df_1 = grouped_weekly.apply(calculate_top_30).reset_index(name='gpr_index_weekly')
    #sentiment_df_2 = grouped_monthly.apply(calculate_top_30).reset_index(name='gpr_index_monthly')
    # Calculate monthly average values of the sentiment
    sentiment_df_2 = sentiment_df_1.groupby(['Year', 'Month'])['gpr_index_weekly'].sum().reset_index()
    sentiment_df_2 = sentiment_df_2.rename(columns={'gpr_index_weekly': 'gpr_index_monthly'})
    
    sentiment_df = pd.merge(sentiment_df_1, sentiment_df_2, on=['Year', 'Month'], how='outer')
    
    return sentiment_df, sentiment_df_2

In [ ]:
input_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\cal_8_geonews.csv'
df_geo = pd.read_csv(input_file_path)

# Step 3: Call the function
sentiment_df, sentiment_df_monthly = sentiment_cals(df_geo)

input_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_output_folder_16Dec\GPR_downloaded_data.csv'
df_index_data = pd.read_csv(input_file_path)

# Step 4: Output the dataset
sentiment_df_2 = pd.merge(sentiment_df, df_index_data, on=['Year', 'Month'], how='outer')
output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\geonews_index_zhi_da.csv'
sentiment_df_2.to_csv(output_file_path, index=False)
sentiment_df_2

# Step 4: Output the dataset
sentiment_df_3 = pd.merge(sentiment_df_monthly, df_index_data, on=['Year', 'Month'], how='outer')
output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\geonews_index_zhi_da_monthly.csv'
sentiment_df_3.to_csv(output_file_path, index=False)
sentiment_df_3

# correlation matrix year by year

In [ ]:
input_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\geonews_index_zhi_da_monthly.csv'
df = pd.read_csv(input_file_path)

# Group by Year and calculate correlation
yearly_corr = df.groupby('Year').apply(lambda group: group[['gpr_index_monthly', 'GPR_paper']].corr().iloc[0, 1])

overall_corr = df[['gpr_index_monthly', 'GPR_paper']] .corr()

# Reset index for better readability (optional)
yearly_corr = yearly_corr.reset_index(name='Correlation')

print(yearly_corr)
print(overall_corr)

import seaborn as sns
import matplotlib.pyplot as plt

# Create a heatmap for overall correlation
plt.figure(figsize=(5, 4))
sns.heatmap(overall_corr, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Overall Correlation Matrix')
plt.show()

# Bar plot for yearly correlation
plt.figure(figsize=(10, 6))
sns.barplot(x='Year', y='Correlation', data=yearly_corr, palette='coolwarm')
plt.title('Yearly Correlation between gpr_index_monthly and GPR_paper')
plt.show()


## Code to get top 30 keywords for each year and month combination

In [10]:
input_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\cal_8_geonews.csv'
df = pd.read_csv(input_file_path)

# create a function to get top 30 keywords
def get_top_keywords(group):
    return group.nsmallest(30, 'rank')

# group by year and month and apply function to dataset
top_keywords = df.groupby(['Year', 'Month'], group_keys=False).apply(get_top_keywords)

# reset index for better readability
top_keywords.reset_index(drop=True, inplace=True)

# Output dataset to output folder
output_file_path = r'C:\Users\em18921\Documents\Sentinels_indicator\Geopolitiks_news\Geonews_output_files\top_30_keywords.csv'
top_keywords.to_csv(output_file_path, index=False)

top_keywords

C:\Users\em18921\AppData\Local\Temp\ipykernel_3500\22740021.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_keywords = df.groupby(['Year', 'Month'], group_keys=False).apply(get_top_keywords)


,date,Year,Month,Day,batch_number,keyword_name,SVI,week_number,time_series_period_week,delta_SVI,...,std_residual,normalize_residual,snp500_tot_return,weekly_index_return_deci,weekly_index_return_prcnt,index_name,snp_Volatility,t_stat,date_calculated,rank
0,20/01/2008,2008,1,20,apidata_131,alliance,54,3.0,3.0,54.0,...,17.002215,2.842828,1664.57,0.016134,1.613415,snp_500,NaN,NaN,2008-01-20,NaN
1,27/01/2008,2008,1,27,apidata_131,alliance,41,4.0,4.0,-13.0,...,17.002215,-0.799202,1667.26,0.001616,0.161603,snp_500,0.006240,NaN,2008-01-27,NaN
2,6/01/2008,2008,1,6,apidata_112,assassination,10,1.0,1.0,-27.0,...,16.199098,-1.690048,1618.05,0.012078,1.207826,snp_500,NaN,NaN,2008-01-06,NaN
3,13/01/2008,2008,1,13,apidata_112,assassination,10,2.0,2.0,0.0,...,16.199098,-0.024682,1638.14,0.012416,1.241618,snp_500,NaN,NaN,2008-01-13,NaN
4,20/01/2008,2008,1,20,apidata_112,assassination,4,3.0,3.0,-6.0,...,16.199098,-0.396467,1664.57,0.016134,1.613415,snp_500,NaN,NaN,2008-01-20,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6085,17/11/2024,2024,11,17,netdata_47,tariff news,27,46.0,881.0,-18.0,...,3.565870,-3.930890,7390.61,0.022142,2.214228,snp_500,0.054145,NaN,2024-11-17,6.0
6086,24/11/2024,2024,11,24,netdata_47,tariff news,100,47.0,882.0,73.0,...,3.565870,3.683638,7336.76,-0.007286,-0.728627,snp_500,0.054336,NaN,2024-11-24,6.0
6087,3/11/2024,2024,11,3,netdata_37,troops at the border,52,44.0,879.0,7.0,...,9.889271,0.705091,6734.84,-0.056191,-5.619124,snp_500,0.039162,NaN,2024-11-03,7.0
6088,10/11/2024,2024,11,10,netdata_37,troops at the border,37,45.0,880.0,-15.0,...,9.889271,-1.521825,7230.51,0.073598,7.359789,snp_500,0.053396,NaN,2024-11-10,7.0
